In [1]:
import sys
import os

# Add the scripts directory to the system path
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [21]:
from pathlib import Path
from data_processor import count_page_breaks, create_consolidated_report, generate_report
from data_downloader import download_data
from results_repo import ConsultingResultRepo
from webdav4.fsspec import WebdavFileSystem
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd

In [3]:
os.environ['DB_WRITEMODE'] = '1'

In [4]:
raw_data_path = 'raw_data/'
data_consulting_stats_path = 'data_consulting_stats'

In [5]:
download_data()

In [6]:
reports = [file for file in Path(raw_data_path).iterdir() if file.is_file()]
consolidated_report = create_consolidated_report(reports)

In [7]:
# complete list
consolidated_list = consolidated_report.split('\n')
len(consolidated_list)

3865

In [8]:
# removing empty strings from list
consolidated_list_empty_string_remove = list(filter(None, consolidated_list))
len(consolidated_list_empty_string_remove)

1479

In [15]:
# finding scholar names (still takes some strings with 'Meeting topic' removing them is taken care of with the below code too)
scholar_names = []

for i in range(len(consolidated_list_empty_string_remove) - 1):  
    if "Scholar(s):" in consolidated_list_empty_string_remove[i]:
        next_element = consolidated_list_empty_string_remove[i + 1]
        if not (next_element.startswith("Meeting topic") or next_element.startswith("\t")):
            scholar_names.append(next_element)

len(scholar_names)

55

In [16]:
df = pd.DataFrame({'scholars': scholar_names})

In [19]:
num_unique_scholars = len(df.scholars.unique())

In [20]:
num_unique_scholars

22

### Uploading results on Sciebo

In [22]:
load_dotenv()
USR = os.getenv('USR')
fs = WebdavFileSystem("https://uni-bonn.sciebo.de/public.php/webdav", auth=(USR, ""))

In [26]:
# Connecting to repo
repo = ConsultingResultRepo.connect(fs)
repo.list()

[]

In [27]:
# Pulling existing results from sciebo
repo.pull()

In [31]:
# updating number of unique researchers
repo.put(
    short_name='num_researchers',
    name='Number of researchers',
    value=num_unique_scholars,
    units='Researcher',
    display_units='Researcher'
)

In [34]:
# pushing to remote
repo.push()